In [3]:
import numpy as np
import matplotlib.pyplot as plt


Bad key mathtext.fallback_to_cm in file /home/diego/.config/matplotlib/stylelib/belle2_serif.mplstyle, line 27 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.10.0/lib/matplotlib/mpl-data/matplotlibrc
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /home/diego/.config/matplotlib/stylelib/belle2.mplstyle, line 36 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.10.0/lib/matplotlib/mpl-data/matplotlibrc
or from the matplotlib source distribution


In [4]:
import torch
import torch.nn as nn # neural network
import torch.nn.functional as F # activation functions

In [5]:
import sys
import os

sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../utils"))

from utils.get_images import get_images

In [7]:
mnist_path = '../mnist/'

x_train_num, y_train_num, x_test_num, y_test_num = get_images(mnist_path)

# se convierten las imagenes en vectores
x_train = x_train_num[:50000].reshape(50000, -1).astype(np.float32)
y_train = y_train_num[:50000].reshape(50000, 1)

x_val = x_train_num[50000:].reshape(10000, -1).astype(np.float32)
y_val = y_train_num[50000:].reshape(10000, 1)

x_test = x_test_num.copy().reshape(10000, -1).astype(float)
y_test = y_test_num.copy().reshape(10000, 1)

In [9]:
# función que normaliza y nos permite tener una media de 0 y desviación estándar de 1
def normalise(x_mean, x_std, x_data):
    return (x_data - x_mean) / x_std

In [10]:
x_mean = x_train.mean()
x_std = x_train.std()

x_train = normalise(x_mean, x_std, x_train)
x_val = normalise(x_mean, x_std, x_val)
x_test = normalise(x_mean, x_std, x_test)

In [11]:
print(x_test.shape)

(10000, 784)


In [12]:
print(y_test.shape)

(10000, 1)


In [24]:
def create_minibatches(x, y, mb_size, shuffle = True):
    '''
    x  #muestras, 784
    y #muestras, 1
    shuffle: mezclar los datos, que ordene de forma aleatorea los datos
    '''
    assert x.shape[0] == y.shape[0], 'Error en cantidad de muestras'
    total_data = x.shape[0]
    if shuffle: 
        idxs = np.arange(total_data)
        np.random.shuffle(idxs)
        x = x[idxs]
        y = y[idxs]  
    return ((x[i:i+mb_size], y[i:i+mb_size]) for i in range(0, total_data, mb_size))

## Pytorch

In [25]:
# convertir los arreglos numpy a Tensores de Pytorch

x_train_tensor = torch.tensor(x_train.copy()) # se copia para no modificar el original y no generar warning de sobreescribir
y_train_tensor = torch.tensor(y_train.copy())

x_val_tensor = torch.tensor(x_val.copy())
y_val_tensor = torch.tensor(y_val.copy())

x_test_tensor = torch.tensor(x_test.copy())
y_test_tensor = torch.tensor(y_test.copy())

# Verificar GPU disponible

In [26]:
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(f"Se esta utilizando {device}")

Se esta utilizando cpu


In [27]:
# Función de accuracy se debe modificar

def accuracy(model, x, y, mb_size):
    num_correct = 0
    num_total = 0
    model.eval() # aquellas capas que se comportan de manera diferente en entrenamiento y evaluación
    model = model.to(device) # donde queremos correr el modelo, si tenemos gpu allí
    with torch.no_grad(): # para que no calcule el gradiente de los parámetros, porque es más eficiente
        for (xi, yi) in create_minibatches(x, y, mb_size):
            xi = xi.to(device, dtype=torch.float32)
            yi = yi.to(device, dtype=torch.long)
            scores = model(xi) # 1, 10 ... mb_size, 10
            _, pred = scores.max(dim=1)
            num_correct += (pred == yi.squeeze()).sum() # squeeze para quitar la dimensión de 1 en el vector yi mb_sizex1
            num_total += pred.size(0)
        return float(num_correct) / num_total

## Loop de entrenamiento

In [28]:
def train(model, optimiser, mb_size, epochs=100):
    model = model.to(device=device)
    for epoch in range(epochs):
        for (xi, yi) in create_minibatches(x_train_tensor, y_train_tensor, mb_size):
            model.train()
            xi = xi.to(device, dtype=torch.float32)
            yi = yi.to(device, dtype=torch.long)
            scores = model(xi)
            # función de costo
            cost = F.cross_entropy(scores, yi.squeeze()) # internamente ya se calcula el softmax
            optimiser.zero_grad() # para que no acumule los gradientes dado que esta dentro de un for el train
            cost.backward()
            optimiser.step() # forma de actualizar los parámetros

        print(f"Epoch {epoch} - Cost: {cost.item()} - Val accuracy: {accuracy(model, x_val_tensor, y_val_tensor, mb_size)}")

## Modelo usando Sequential

In [39]:
hidden1 = 1000
hidden = 1000
learning_rate = 5e-2
epochs = 100
mb_size = 4096 # es bueno en potencias de 2

In [36]:
#model1 = nn.Sequential(nn.Linear(784, hidden1), nn.ReLU(),
#                       nn.Linear(hidden1, hidden), nn.ReLU(),
#                       nn.Linear(hidden, 10)
#                      )

# muchas veces un modelo más grande es más difícil de entrenar y podemos hacer uno más pequeño
# aunque un modelo pequeño no significa mejor, porque puede que no tenga la capacidad de aprender

model1 = nn.Sequential(nn.Linear(784, 10), nn.ReLU(), nn.Linear(10, 10))

# si comento mi modelo anterior, se corre sumando la nueva cantidad de epochs añadidas, al inicio tuve
# 50 epochs y cambié este parámetro a 100 entonces se acumulará la corrida del train a 150 epochs
# si descomento el model1 y lo vuelvo a correr, se reiniciará el contador de epochs a 0, es decir un modelo nuevo

# También, si en las capas intermedias añado más neuronas, el modelo es más complejo y más difícil de sintonizar

In [37]:
optimiser = torch.optim.SGD(model1.parameters(), lr=learning_rate)

In [40]:
train(model1, optimiser, mb_size, epochs)

Epoch 0 - Cost: 0.43019482493400574 - Val accuracy: 0.8916
Epoch 1 - Cost: 0.40504902601242065 - Val accuracy: 0.8921
Epoch 2 - Cost: 0.4682528078556061 - Val accuracy: 0.8929
Epoch 3 - Cost: 0.42822545766830444 - Val accuracy: 0.8928
Epoch 4 - Cost: 0.42836496233940125 - Val accuracy: 0.8938
Epoch 5 - Cost: 0.4132584035396576 - Val accuracy: 0.8949
Epoch 6 - Cost: 0.4283534288406372 - Val accuracy: 0.8943
Epoch 7 - Cost: 0.48783594369888306 - Val accuracy: 0.8961
Epoch 8 - Cost: 0.3934120535850525 - Val accuracy: 0.8955
Epoch 9 - Cost: 0.3931410014629364 - Val accuracy: 0.8955
Epoch 10 - Cost: 0.4372955560684204 - Val accuracy: 0.8962
Epoch 11 - Cost: 0.4120085835456848 - Val accuracy: 0.8967
Epoch 12 - Cost: 0.41573280096054077 - Val accuracy: 0.8973
Epoch 13 - Cost: 0.4359157085418701 - Val accuracy: 0.8974
Epoch 14 - Cost: 0.43391698598861694 - Val accuracy: 0.898
Epoch 15 - Cost: 0.44330212473869324 - Val accuracy: 0.8986
Epoch 16 - Cost: 0.3930099904537201 - Val accuracy: 0.8989


In [41]:
accuracy(model1, x_test_tensor, y_test_tensor, mb_size)

0.9082

In [42]:
# Este accuracy con un modelo sencillo en Pytorch sin tunning